In [ ]:
import cv2
import matplotlib.pyplot as plt

# SURF descriptor

## Overview
The SURF algorithm is based on the same principles and steps as SIFT; but details in each step are different. <br>
The algorithm is composed of 2 main parts:<br>
Part 1: interest point detection<br>
Part 2: local neighborhood description<br>

<br><strong>1. Interesting point detection</strong><br>

1.1. Fast-Hessian Detector<br><br>
SURF uses the Hessian matrix because of its good performance in computation time and accuracy. Rather than using a <br>
different measure for selecting the location and the scale (Hessian-Laplace detector). Given a point x = (x, y) in an <br>
image I, the Hessian matrix H(x, σ) in x at scale σ is defined as follows: <br><br>
\begin{equation*}
H(x, \sigma) = 
\begin{bmatrix}
L_{xx}(x, \sigma) & L_{xy}(x, \sigma) \\
L_{xy}(x, \sigma) & L_{yy}(x, \sigma)
\end{bmatrix}
\end{equation*}

where 

\begin{align*} 
L_{xx}(x, \sigma) 
\end{align*}
is the convolution of the Gaussian second order derivative: 
\begin{align*} 
\frac{d^2}{dx^x} G(\sigma) \text{ with the image I in point x }
\end{align*}

In order to calculate the determinant of the Hessian matrix, first we need to apply convolution with Gaussian kernel, then <br>
second-order derivative. In SIFT, Lowe approximated Laplacian of Gaussian with Difference of Gaussian for finding scale-space. <br>
SURF goes a little further and approximates LoG with Box Filter. Below image shows a demonstration of such an approximation:<br><br>
<img src="surf_boxfilter.jpg"><br><br>
The crude approximations are valuable because they can be very quickly run at any scale due to the use of an integral image.<br><br>
<img src="masks.jpg"><br>
<small>Approximated Gaussian second derivative in y-direction and xy-direction, used for the SURF detector.</small><br><br><br>

1.2. Scale-space representation<br><br>
Scale spaces are usually implemented as image pyramids. The images are repeatedly smoothed with a Gaussian and subsequently <br>
sub-sampled in order to achieve a higher level of the pyramid. Due to the use of box filters and integral images, SURF does not <br>
have to iteratively apply the same filter to the output of a previously filtered layer, but instead can apply such filters of any <br>
size at exactly the same speed directly on the original image, and even in parallel (although the latter is not exploited here).<br> 
Therefore, the scale space is analysed by up-scaling the filter size rather than iteratively reducing the image size. Besides, <br>
each kernel need to have an uneven size to have a central pixel. This results in filters of size 9×9, 15×15, 21×21, 27×27, etc..<br>
With this step, extremas can be found out, and they are detected as points of interests.<br>


<br><strong>2. Local neighborhood description</strong><br>

The SURF descriptor is designed to be scale invariant and rotationally invariant.<br>
To ignore scale the descriptor is sampled over a window that is proportional to the window size with which it was detected, <br>
that way if a scaled version of the feature is in another image the descriptor for that feature will be sampled over the same <br>
relative area.<br>

Rotation is handled by finding the dominant direction of the feature and rotating the sampling window to align with that angle.<br>
Once the rotated neighborhood is obtained it is divided up into 16 subsquares, each sub square is again divided into 4 <br>
squares. Derivatives in the x and y directions are taken in these final squares. The descriptor for the sub square is the sum <br>
of the x derivatives over its four quadrants, sum of the absolute values of the x derivatives and similarly for y. The total <br>
descriptor is 4 values per subsquare for a total of 64 values. This vector is normalized to length 1 and is the feature descriptor.<br>
<br><img src="surf_descriptor.jpg"><br>
<small>A graphical representation of the SURF descriptor. This figure taken from the original SURF paper.</small><br><br>
This 16x4 descriptor is a 64 dimensional one, and SURF also supports an extended 128 dimensional descriptor, for cases <br>
requiring a higher distinctiveness in its features.

## OpenCV SURF
OpenCV provides SURF functionalities just like SIFT. We can use SURF.detect(), SURF.compute() etc for finding keypoints and descriptors.

In [ ]:
def apply_SURF_descriptor(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Applying SURF detector
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(gray_img, None)
    print("Number of keypoints: ", len(kp))
    print ("Keypoint description:")
    for p in des:
        print(p)

    # Marking the keypoint on the image using circles
    img=cv2.drawKeypoints(gray_img ,
                            kp ,
                            img ,
                            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    plt.figure(figsize=(20, 20))
    plt.imshow(img)

In [ ]:
img = cv2.imread("../hanoi.jpg")
plt.figure(figsize=(20, 20))
plt.imshow(img)

In [ ]:
apply_SURF_descriptor(img)

<u>Sources:</u><br><br>
https://people.ee.ethz.ch/~surf/eccv06.pdf<br><br>
https://medium.com/data-breach/introduction-to-surf-speeded-up-robust-features-c7396d6e7c4e <br><br>
https://docs.opencv.org/3.4/df/dd2/tutorial_py_surf_intro.html <br><br>
https://courses.cs.washington.edu/courses/cse576/13sp/projects/project1/artifacts/woodrc/index.htm <br><br>
https://zhvillues.tumblr.com/post/119561114181/sift-vs-surf <br><br>